In [1]:
import graphlab as gl

In [1]:
timezone = gl.SFrame.read_csv('../timezonedb.csv/zone.csv')
country = gl.SFrame.read_csv('../timezonedb.csv/country.csv')

NameError: name 'gl' is not defined

In [3]:
timezone = timezone.join(country,on = 'CountryCode')
timezone.head()

Id,CountryCode,Zone,Country
1,AD,Europe/Andorra,Andorra
2,AE,Asia/Dubai,United Arab Emirates
3,AF,Asia/Kabul,Afghanistan
4,AG,America/Antigua,Antigua and Barbuda
5,AI,America/Anguilla,Anguilla
6,AL,Europe/Tirane,Albania
7,AM,Asia/Yerevan,Armenia
8,AO,Africa/Luanda,Angola
9,AQ,Antarctica/McMurdo,Antarctica
10,AQ,Antarctica/Casey,Antarctica


In [4]:
def return_country_from_zone(x):
    d = 'United States'
    for row in timezone:
        if(x in row['Zone']):
            d = row['Country']
            break
    return d
            

In [43]:
#can't get the location on nodes,the reason is that the top influence might not appear in dataset,he is only in our mentions,
#in racism : Blk_voice
def create_user_mentions(sub):
    #source = []
    #target = []
    nodeList = []
    edgelist = []
    for tweet in sub:
        for e in tweet['X1.entities.user_mentions']:
            #source.append(tweet['X1.user.screen_name'])
            nodeList.append(tweet['X1.user.screen_name'])
            #target.append(e['screen_name'])
            nodeList.append(e['screen_name'])
            edgelist.append({'source' : tweet['X1.user.screen_name'],'target' : e['screen_name'],'location' : tweet['X1.user.location'],
                             'timezone' : tweet['X1.user.time_zone']})
    nodeList = list(set(nodeList))
    return edgelist,nodeList

In [44]:
def produce_edges_nodes(filename):
    print '------------ '+filename+ 'Cause ------------'
    tweets = gl.SFrame.read_csv(filename+'_sf.csv',verbose=False)
    print "Total Tweets : %d"%tweets.num_rows()
    #removing tweets that have no mentions in it
    tweets['X1.entities.user_mentions'] = tweets['X1.entities.user_mentions'].astype(list)
    sub = tweets[tweets['X1.entities.user_mentions'] != []]
    print "Total Tweets after removing non-mentions : %d"%sub.num_rows()
    edgeList,nodeList = create_user_mentions(sub)
    edgeList = gl.SFrame(edgeList)
    edgeList = edgeList.unpack('X1')
    edgeList['country'] = edgeList['X1.timezone'].apply(return_country_from_zone)
    nodeListSF = gl.SFrame({'Id' : nodeList,'Label' : nodeList})
    edgeList.export_csv('nodes_edges/'+filename+'_edges.csv')
    nodeListSF.export_csv('nodes_edges/'+filename +'_nodes.csv')
    print '------------ x ------------'
    return True

In [45]:
social_causes = ['childabuse','climate','edu','gender','health','humanrights','hunger','poverty','racism','refugees','terrorism']

for i in range(0,len(social_causes)):
    produce_edges_nodes(social_causes[i])

------------ childabuseCause ------------
Total Tweets : 336
Total Tweets after removing non-mentions : 199
------------ x ------------
------------ climateCause ------------
Total Tweets : 949
Total Tweets after removing non-mentions : 719
------------ x ------------
------------ eduCause ------------
Total Tweets : 963
Total Tweets after removing non-mentions : 479
------------ x ------------
------------ genderCause ------------
Total Tweets : 663
Total Tweets after removing non-mentions : 301
------------ x ------------
------------ healthCause ------------
Total Tweets : 2627
Total Tweets after removing non-mentions : 589
------------ x ------------
------------ humanrightsCause ------------
Total Tweets : 265
Total Tweets after removing non-mentions : 177
------------ x ------------
------------ hungerCause ------------
Total Tweets : 5220
Total Tweets after removing non-mentions : 3460
------------ x ------------
------------ povertyCause ------------
Total Tweets : 197
Total Tw